# Prototype of lipreading pipeline

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

import random
import math
import time

# model file, encoder, decoder and seqtoseq
from model_temp import *
# utils file
from utils import *
# Get landmark using vocadataset.py
from data.vocaset import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [38]:
# Get landmark from vocadaset class
#trainset = vocadataset("train", landmark=True)
trainset = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]

trainloader = DataLoader(trainset, batch_size=20, collate_fn=collate_fn, shuffle=True)

#landmarks, len_landmark, label, len_label = next(iter(trainloader))


In [3]:
vocabulary = create_vocabulary(blank="@")

In [ ]:
# Create a mapping from characters to indices
char_to_index = {char: index for index, char in enumerate(vocabulary)}

In [4]:
# Convert the sequence and target to indices
#sequence_indices = [char_to_index[char] for char in sequence]

label_t = char_to_index_batch(label, vocabulary)

#target_indices = [char_to_index[char] for char in labels]
#target_tensor = torch.tensor(target_indices)

In [32]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder2(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)



In [ ]:
print(torch.reshape(landmarks[0], (landmarks[0].shape[0], landmarks[0].shape[1]*landmarks[0].shape[2]))[None,:,:].shape)
print(len_landmark[0][None, None].shape)

In [ ]:
landmarks.shape

In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10000
for epoch in range(num_epochs):
    
    for i in range(landmarks.shape[0]):
        landmarks[i] = landmarks[i].to(device) 
        label_t[i] = label_t[i].to(device)
        optimizer.zero_grad()
        target_tensor = label_t[i]

        

        reshaped_landmark = torch.reshape(landmarks[i], (landmarks[i].shape[0], landmarks[i].shape[1]*landmarks[i].shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark[i] = len_landmark[i].to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



In [20]:
nw = landmarks[0][:len_landmark[0], :, :].clone().detach().requires_grad_(True)
nw.shape

torch.Size([269, 36, 3])

In [31]:
nw_l = label_t[0][:len_label[0]].clone().detach()


In [29]:
label_t[0]

tensor([ 2, 28,  4, 19,  2,  3, 28,  4,  9,  2, 13, 13,  6, 15,  8,  6,  5, 28,
        14,  6, 31, 28,  3, 22, 21, 28,  2, 28, 18, 22, 10,  4, 12, 28, 20, 21,
         2,  3, 28, 23,  2, 15, 18, 22, 10, 20,  9,  6,  5, 28,  9, 10, 14, 29,
        29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29, 29])

In [ ]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2000
for _ in range (16):
    landmarks, len_landmark, label, len_label = next(iter(trainloader))

    for epoch in range(num_epochs):
        
        for i in range(landmarks.shape[0]):
            landmarks[i] = landmarks[i].to(device) 
            label_t[i] = label_t[i].to(device)
            optimizer.zero_grad()


            #remove padding from dataloader
            nw = landmarks[i][:len_landmark[i], :, :].clone().detach().requires_grad_(True)
            nw_l = label_t[i][:len_label[i]].clone().detach()
            
            target_tensor = nw_l
            reshaped_landmark = torch.reshape(nw, (nw.shape[0], nw.shape[1]*nw.shape[2]))
            reshaped_landmark = reshaped_landmark.to(device)
            target_tensor = target_tensor.to(device)
            len_landmark[i] = len_landmark[i].to(device) 
            output = model(reshaped_landmark[None, : , :], len_landmark[i][None])
            output = output.permute(1,0,2)
            input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
            target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
            
            loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor, input_lengths, target_lengths[0])
            loss.backward()
            optimizer.step()

            e = torch.argmax(output, dim=2).squeeze(1)
            output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)
            """if(epoch + 1) % 100 == 0:
                f = open("prova_.txt", "a")
                f.write(str(nw_l.tolist())+"\n")
                f.write(output_sequence+"\n")
                f.close() 
        """
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
            #e = torch.argmax(output, dim=2).squeeze(1)
            #output_sequence = ''.join([vocabulary[index] for index in e])
            #print(output_sequence)



In [32]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########

#trainset = vocadataset("train", landmark=True)
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)
#landmark, labels = trainset[0]
trainset_ = trainset_.to(device)
trainloader_ = DataLoader(trainset_, batch_size=1, collate_fn=collate_fn, num_workers=8)


###########à

AttributeError: 'vocadataset' object has no attribute 'to'

In [28]:
INPUT_DIM = 36*3
INPUT_DIM = 36*3
HID_DIM = 64
output_dim = len(vocabulary)

model = only_Decoder2(INPUT_DIM, HID_DIM, 2, len(vocabulary)).to(device)

In [9]:
trainset_[0][0].shape

torch.Size([243, 36, 3])

In [12]:
torch.tensor(trainset_[0][0].shape[0]).shape

torch.Size([])

In [31]:
# Define the CTC loss function
ctc_loss = nn.CTCLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)


###########
trainset_ = vocadataset("train", landmark=True, mouthOnly=True)


# Training loop
num_epochs = 10000
for epoch in range(num_epochs):

    for i  in range(10):
        optimizer.zero_grad()

        landmarks = trainset_[i][0].to(device) 
        
        label_t = char_to_index_batch(trainset_[i][1], vocabulary)
        label_t = label_t.to(device)
        target_tensor = label_t
        reshaped_landmark = torch.reshape(landmarks, (landmarks.shape[0], landmarks.shape[1]*landmarks.shape[2]))
        reshaped_landmark = reshaped_landmark.to(device)
        target_tensor = target_tensor.to(device)
        len_landmark = torch.tensor(landmarks.shape[0]).to(device) 
        output = model(reshaped_landmark[None, : , :], len_landmark[None])
        output = output.permute(1,0,2)
        input_lengths = torch.full((1,), output.size(0), dtype=torch.long)
        target_lengths = torch.full((target_tensor.size(0),), target_tensor.size(0), dtype=torch.int32)
        
        loss = ctc_loss(torch.nn.functional.log_softmax(output, dim=2), target_tensor.squeeze(), input_lengths, target_lengths[0][None])
        loss.backward()
        optimizer.step()

        e = torch.argmax(output, dim=2).squeeze(1)
        output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)
        if(epoch + 1) % 100 == 0:
            f = open("prova_.txt", "a")
            f.write(label[i]+"\n")
            f.write(output_sequence+"\n")
            f.close() 
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")
        #e = torch.argmax(output, dim=2).squeeze(1)
        #output_sequence = ''.join([vocabulary[index] for index in e])
        #print(output_sequence)



Epoch [10/10000], Loss: 3.023833990097046


KeyboardInterrupt: 

In [30]:
target_tensor.shape

torch.Size([53, 1])

In [40]:
torch.save(model.state_dict(), "models/model20sent_3.pt")